# Setup Cube 3D

In [1]:
!git clone https://github.com/Roblox/cube

Cloning into 'cube'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 82 (delta 15), reused 18 (delta 9), pack-reused 52 (from 1)
Receiving objects: 100% (82/82), 25.14 MiB | 31.20 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [2]:
%cd /content/cube

/content/cube


In [3]:
!huggingface-cli download Roblox/cube3d-v0.1 --local-dir ./model_weights

Fetching 6 files:   0% 0/6 [00:00<?, ?it/s]Downloading 'shape_tokenizer.safetensors' to 'model_weights/.cache/huggingface/download/hvCscYm1hYro8jq0ze4Xbztgnro=.9c9eb6ba7e160355daf55cf67d8ccb7e490ca3de08eebd2efc84a9f035c6b8f9.incomplete'

logo.png:   0% 0.00/74.6k [00:00<?, ?B/s]Downloading '.gitattributes' to 'model_weights/.cache/huggingface/download/wPaCkH-WbT7GsmxMKKrNZTV4nSM=.30f47c36eac0a062259655fa315e6778e2ba4129.incomplete'
logo.png: 100% 74.6k/74.6k [00:00<00:00, 1.25MB/s]
Download complete. Moving file to model_weights/assets/images/logo.png

shape_tokenizer.safetensors:   0% 0.00/1.09G [00:00<?, ?B/s]

shape_gpt.safetensors:   0% 0.00/7.17G [00:00<?, ?B/s]


README.md: 100% 1.37k/1.37k [00:00<00:00, 8.75MB/s]
Download complete. Moving file to model_weights/README.md



.gitattributes: 100% 1.75k/1.75k [00:00<00:00, 14.4MB/s]
Download complete. Moving file to model_weights/.gitattributes
Fetching 6 files:  17% 1/6 [00:00<00:02,  2.40it/s]


greyscale_512.gif:   0% 0.00/8.10M 

In [4]:
!pip install trimesh omegaconf warp-lang transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.9/708.9 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 MB 7.2 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144555 sha256=49a1caff39bdb240ad5e4506dbaa73d98efd4085d8f5e9fdf3a8ef2f9786b911
  Stored in directory: /root/.cache/pip/wheels/1a/97/32/461f837398029ad76911109f07047fde1d7b661a147c7c56d1
Successfully built antlr4-python3-runtime


In [2]:
%cd /content/cube

/content/cube


# Create Inference Engine

In [3]:
from cube3d.inference.engine import Engine
import torch
# load ckpt
config_path = "cube3d/configs/open_model.yaml"
gpt_ckpt_path = "model_weights/shape_gpt.safetensors"
shape_ckpt_path = "model_weights/shape_tokenizer.safetensors"
engine = Engine(
    config_path,
    gpt_ckpt_path,
    shape_ckpt_path,
    device=torch.device("cuda"),
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

# Generate a Mesh from text

In [4]:
input_prompt = "vintage couch"
# Use a lower resolution_base to accomodate limited GPU VRAM on Colab notebooks
mesh_v_f = engine.t2s([input_prompt], use_kv_cache=True, resolution_base=5.0)

extracting geometry: 100%|██████████| 1/1 [00:00<00:00,  2.43chunk/s]


Warp 1.6.2 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.6.2


# Visualize the output

In [6]:
# Visualize the model
import plotly.graph_objects as go
vertices = mesh_v_f[0][0]
faces = mesh_v_f[0][1]
fig = go.Figure(
    data=[
        go.Mesh3d(
            x=vertices[:,0],
            y=vertices[:,1],
            z=vertices[:,2],
            i=faces[:,0],
            j=faces[:,1],
            k=faces[:,2],
            opacity=1.0)
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()